In [2]:
from gensim.corpora.wikicorpus import WikiCorpus
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from pprint import pprint
import multiprocessing
from tqdm import tqdm

In [9]:
from tqdm import tqdm

In [25]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [ ]:
import gensim.downloader as api
data = api.load("wiki-english-20171001")

In [5]:
wiki = WikiCorpus("enwiki-latest-pages-articles.xml.bz2", dictionary={})

In [16]:
class TaggedWikiDocument(object):
    def __init__(self, wiki):
        self.wiki = wiki
        self.wiki.metadata = True
    def __iter__(self):
        for content, (page_id, title) in self.wiki.get_texts():
            yield TaggedDocument([c for c in content], [title])

In [17]:
documents = TaggedWikiDocument(wiki)

In [18]:
pre = Doc2Vec(min_count=0)
pre.scan_vocab(documents)

(365218104, 214876)

Process InputQueue-192:
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/local/lib/python3.6/dist-packages/gensim/utils.py", line 1289, in run
    wrapped_chunk = [list(chunk)]
  File "/usr/local/lib/python3.6/dist-packages/gensim/corpora/wikicorpus.py", line 676, in <genexpr>
    (text, title, pageid, tokenization_params)
  File "/usr/local/lib/python3.6/dist-packages/gensim/corpora/wikicorpus.py", line 421, in extract_pages
    for elem in elems:
  File "/usr/local/lib/python3.6/dist-packages/gensim/corpora/wikicorpus.py", line 406, in <genexpr>
    elems = (elem for _, elem in iterparse(f, events=("end",)))
  File "/usr/lib/python3.6/xml/etree/ElementTree.py", line 1223, in iterator
    data = source.read(16 * 1024)
  File "/usr/lib/python3.6/bz2.py", line 182, in read
    return self._buffer.read(size)
  File "/usr/lib/python3.6/_compression.py", line 68, in readinto
    data = self.read(l

In [ ]:
for num in range(0, 20):
    print('min_count: {}, size of vocab: '.format(num), pre.scale_vocab(min_count=num, dry_run=True)['memory']['vocab']/700)

In [29]:
cores = 8

models = [
    # PV-DBOW 
    Doc2Vec(dm=0, dbow_words=1, vector_size=200, window=8, min_count=19, workers=cores),
    # PV-DM w/average
    Doc2Vec(dm=1, dm_mean=1, vector_size=200, window=8, min_count=19, workers=cores),
]

In [ ]:
models[0].build_vocab(corpus_file="./wiki-english-20171001.gz")
print(str(models[0]))
models[1].reset_from(models[0])
print(str(models[1]))

Doc2Vec(dbow+w,d200,n5,w8,mc19,s0.001,t8)


In [ ]:
for model in models:
    model.train(documents, total_examples=model.corpus_count, epochs=10)

In [36]:
models[0].save("Doc2Vec_enwiki_DBOW_200_8_10.model")
models[1].save("Doc2Vec_enwiki_DM_200_8_10.model")